In [99]:
import vectorbt as vbt
import pandas as pd
from datetime import datetime
import numpy as np
from numba import njit
from tqdm import tqdm

import plotly.graph_objects as go
import plotly.express as px
import typing as tp

In [100]:
df = pd.DataFrame(vbt.YFData.download('BTC-USD').get('Close'))
df = df[df.index>'2023-01-01']
df['entries'] = [True if i % 10 == 0 else False for i in range(len(df))]
df['exits'] = False

/Users/k/miniconda3/envs/RAJ/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



In [101]:
length = len(df)

In [102]:
df['entries'] = np.random.rand(length) < 0.55   # Randomly 15% of rows True

In [103]:
df['entries'].value_counts()


entries
True     342
False    307
Name: count, dtype: int64

In [104]:
# AdjustSLContext(
#     i,
#     col,
#     position_now,
#     val_price_now,
#     init_i,
#     init_price,
#     curr_i,
#     curr_price,
#     curr_stop,
#     curr_trail
# )

In [105]:
@njit
def adjust_sl_func_nb(c, *args):
    """Placeholder function that returns the initial stop-loss value and trailing flag."""
    
    if c.curr_price > 1.20 * c.curr_stop:
        STOP = 0.03
        TRAIL = True
    return STOP, TRAIL

portfolio = vbt.Portfolio.from_signals(
    
    close = df.Close,  # Ceny zamknięcia
    entries = df.entries,  # Sygnały wejścia
    exits = df.exits,  # Sygnały wyjścia
    sl_stop = 0.1,  # StopLoss w postaci procentowej
    sl_trail=False,
    adjust_sl_func_nb=adjust_sl_func_nb,
    size = 1000, 
    init_cash = 100000,
    fees=0.002
             
    
    
)

# Note that 0.01 = 1%.

In [107]:
portfolio.total_return()

1.5947911700865582

In [108]:
print(portfolio.stats())
portfolio.plot()


Start                         2023-01-02 00:00:00+00:00
End                           2024-10-11 00:00:00+00:00
Period                                649 days 00:00:00
Start Value                                    100000.0
End Value                                 259479.117009
Total Return [%]                             159.479117
Benchmark Return [%]                         277.478698
Max Gross Exposure [%]                            100.0
Total Fees Paid                            51798.197046
Max Drawdown [%]                              24.109565
Max Drawdown Duration                 184 days 00:00:00
Total Trades                                         65
Total Closed Trades                                  64
Total Open Trades                                     1
Open Trade PnL                              10710.97609
Win Rate [%]                                       37.5
Best Trade [%]                                58.677857
Worst Trade [%]                               -6

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a829ea8c-2bd6-43a3-8b3c-51ff8af1deaa',
              'x': array([datetime.datetime(2023, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 1, 3, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 10, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 11, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([16688.47070312, 16679.85742188, 16863.23828125, ..., 60582.1015625 ,
                

In [ ]:
'''Start: Data rozpoczęcia inwestycji (2023-01-02).
End: Data zakończenia inwestycji (2024-10-11).
Period: Czas trwania inwestycji (649 dni).
Start Value: Wartość początkowa portfela (100,000 USD).
End Value: Wartość końcowa portfela (266,794.71 USD).
Total Return [%]: Całkowity zwrot z inwestycji w procentach (166.79%). Oznacza, że wartość portfela wzrosła o 166.79% w analizowanym okresie.
Benchmark Return [%]: Zwrot z benchmarku w tym samym okresie (277.19%). Benchmark to punkt odniesienia, z którym porównujesz wyniki swojego portfela.
Max Gross Exposure [%]: Maksymalne narażenie portfela (100%). Oznacza, że w pewnym momencie portfel był w pełni narażony na ryzyko.
Total Fees Paid: Całkowite opłaty zapłacone w trakcie inwestycji (32,697.94 USD). Wskazuje, ile zostało wydane na opłaty transakcyjne.
Max Drawdown [%]: Maksymalne spadki wartości portfela (37.51%). Mierzy największy procentowy spadek wartości od szczytu do dna w trakcie okresu.
Max Drawdown Duration: Czas trwania maksymalnego spadku (221 dni). Okres, przez który portfel był w stanie spadkowym.
Total Trades: Łączna liczba przeprowadzonych transakcji (43).
Total Closed Trades: Liczba zakończonych transakcji (42).
Total Open Trades: Liczba otwartych transakcji (1).
Open Trade PnL: Zysk/strata z otwartej transakcji (-533.59 USD). Wartość zysku/straty dla aktualnie otwartej pozycji.
Win Rate [%]: Procent udanych transakcji (45.24%). Wskaźnik mówiący, jak wiele transakcji zakończyło się zyskiem.
Best Trade [%]: Najlepsza transakcja w procentach (59.25%). Maksymalny zysk uzyskany z pojedynczej transakcji.
Worst Trade [%]: Najgorsza transakcja w procentach (-9.85%). Maksymalna strata uzyskana z pojedynczej transakcji.
Avg Winning Trade [%]: Średni zysk z udanych transakcji (11.53%).
Avg Losing Trade [%]: Średnia strata z nieudanych transakcji (-3.99%).
Avg Winning Trade Duration: Średni czas trwania udanych transakcji (13 dni 18:56:50).
Avg Losing Trade Duration: Średni czas trwania nieudanych transakcji (4 dni 22:57:23).
Profit Factor: Wskaźnik rentowności, obliczany jako stosunek zysków do strat (1.90). Oznacza, że zyski były prawie dwa razy większe niż straty.
Expectancy: Oczekiwana wartość transakcji (3984.01 USD). Mierzy średni zysk na transakcję, uwzględniając zarówno zyski, jak i straty.
Sharpe Ratio: Wskaźnik mierzący, jak zyskowne były inwestycje w stosunku do ryzyka (1.60). Wartość powyżej 1 sugeruje, że portfel dobrze radzi sobie w stosunku do ryzyka.
Calmar Ratio: Mierzy zwrot w stosunku do maksymalnego spadku (1.96). Im wyższy, tym lepiej.
Omega Ratio: Wskaźnik rentowności, porównujący zyski do strat w różnych scenariuszach (1.38). Wyższe wartości wskazują na lepszą wydajność.
Sortino Ratio: Mierzy zysk w stosunku do ryzyka strat (2.78). Wartość powyżej 2 jest uważana za bardzo dobrą.
'''